In [1]:
from itertools import repeat
import numpy as np
import seaborn as sb
import matplotlib.pyplot as pp
import pandas as pd
from PIL import Image # Importamos la clase Image de la librería **pillow**
from matplotlib import image
import os
from os import listdir
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow_model_optimization as tfmot

import keras
from keras.models import Sequential #RNA que tiene conexion hacia adelante
from keras.layers import Dense, Dropout, Flatten, MaxPooling2D, LeakyReLU #Capas de RNA
import keras.optimizers as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import confusion_matrix
%matplotlib inline

Using TensorFlow backend.
/Users/jessica/opt/anaconda3/envs/IA2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jessica/opt/anaconda3/envs/IA2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jessica/opt/anaconda3/envs/IA2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jess

ModuleNotFoundError: No module named 'tensorflow_model_optimization'

In [5]:


#Cargamos las categorias en orden 
listaCategorias = os.listdir('./image-dataset-proyecto-integrador-interciclo')
listaCategorias.sort(key = str.lower)
categorias = pd.DataFrame(listaCategorias)
print('Total de Categorias ::::',len(categorias))

categorias



Total de Categorias :::: 6


,0
0,bosque
1,calle
2,edificios
3,glacial
4,mar
5,montaña


In [2]:
#librerias
import numpy as np
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [15]:
import re

In [3]:
IMG_WIDTH=200
IMG_HEIGHT=200
batch_size=4
img_dir =  r'image-dataset-proyecto-integrador-interciclo'
dirname = os.path.join(os.getcwd(), img_dir)
imgpath = dirname + os.sep 
 
images = []
directories = []
dircount = []
prevRoot=''
cant=0
 
print("leyendo imagenes de ",imgpath)
for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            if len(image) ==150:
                cant=cant+1
                images.append(image)
                b = "Leyendo..." + str(cant)
                print (b, end="\r")
                if prevRoot !=root:
                    print(root, cant)
                    prevRoot=root
                    directories.append(root)
                    dircount.append(cant)
                    cant=0
dircount.append(cant)
dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))


leyendo imagenes de  /Users/jessica/Downloads/image-dataset-proyecto-integrador-interciclo/
/Users/jessica/Downloads/image-dataset-proyecto-integrador-interciclo/bosque 1
/Users/jessica/Downloads/image-dataset-proyecto-integrador-interciclo/mar 2263
/Users/jessica/Downloads/image-dataset-proyecto-integrador-interciclo/edificios 2270
/Users/jessica/Downloads/image-dataset-proyecto-integrador-interciclo/calle 2190
/Users/jessica/Downloads/image-dataset-proyecto-integrador-interciclo/montaña 2381
/Users/jessica/Downloads/image-dataset-proyecto-integrador-interciclo/glacial 2495
Directorios leidos: 6
Imagenes en cada directorio [2264, 2270, 2190, 2381, 2495, 2386]
suma Total de imagenes en subdirs: 13986


In [4]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))
 
categorias=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    categorias.append(name[len(name)-1])
    indice=indice+1

y = np.array(labels)

#X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
X=np.array(images, dtype=np.uint8)
# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Cantidad etiquetas creadas:  13986
0 bosque
1 mar
2 edificios
3 calle
4 montaña
5 glacial
Total number of outputs :  6
Output classes :  [0 1 2 3 4 5]


In [5]:
#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)


Training data shape :  (11188, 150, 150, 3) (11188,)
Testing data shape :  (2798, 150, 150, 3) (2798,)
Original label: 2
After conversion to one-hot: [0. 0. 1. 0. 0. 0.]
(8950, 150, 150, 3) (2238, 150, 150, 3) (8950, 6) (2238, 6)


In [ ]:
#Creacion de la red neuronal tipo secuencial
modelo = Sequential()

# Nuestra primera capa va a ser una convolucion que va a tener el numero de filtros que definimos, al igual que las dimensiones
#de trabajo de las imagenes a procesar y una activacion tipo relu
modelo.add(Conv2D(150,(3,3),padding ="same",activation='relu',input_shape=(150, 150,3)))
#Esta capa aplica un filtro que reduce la dimensionalidad de la imagen, esto es, la hace más pequeña.
modelo.add(MaxPooling2D(pool_size=(2, 2)))
modelo.add(Conv2D(64,(2,2),activation='relu',padding ="same"))
modelo.add(MaxPooling2D(pool_size=(2, 2)))
#Aplanar la informacion, una dimension que contiene toda la informacion de nuestra red neuronal
modelo.add(Flatten())
#Añade una capa que estara conectada a las capas anteriores
modelo.add(Dense(256, activation='relu'))
# Durante el entrenamiento se apaga cierto numero de nueronas para evitar sobreajustar los datos
modelo.add(Dropout(0.5))
modelo.add(Dense(100, activation='relu'))
modelo.add(Dropout(0.5))
# softmas: Valores de porcentaje de probabilidad de determinacion de una categoria
modelo.add(Dense(6, activation='softmax'))

modelo.summary()
#optimizar el procesamiento de los datos, funcion de perdida, para ver q tan bien o mal esta aprendiendo la red neural
modelo.compile(loss='categorical_crossentropy',optimizer='nadam',metrics=['accuracy'])
#Entrenamiento de la red nueronal
historial= modelo.fit(train_X, train_label, 
                          batch_size=64,epochs=50,
                          verbose=1,
                          validation_data=(valid_X, valid_label))